# Задача
На основе предоставленных данных необходимо получить итоговый отчет, который содержит следующую информацию для каждого активного клиента за 2024 год:

-	Общую сумму транзакций по типам операций.
-	Среднюю сумму транзакций по типам операций.
-	Количество транзакций по типам операций.
-	Дату последней транзакции.

## Исходные данные:
1. Таблица `transactions` с информацией о транзакциях:

-	transaction_id — уникальный идентификатор транзакции.
-	client_id — идентификатор клиента.
-	transaction_date — дата транзакции.
-	amount — сумма транзакции.
-	currency — валюта транзакции.
-	transaction_type — тип транзакции ("deposit" или "withdrawal").

2. Таблица `clients` с информацией о клиентах:

-	client_id — идентификатор клиента.
-	client_name — имя клиента.
-	registration_date — дата регистрации клиента.
-	client_status — статус клиента ("active" или "inactive").	



In [1]:
# Импортируем необходимые библиотеки
import pandas as pd

In [2]:
# Выгружаем данные
data_transactions = {
    'transaction_id': [1, 2, 3, 4, 5, 6, 7, 8],
    'client_id': [101, 102, 101, 103, 102, 101, 103, 102],
    'transaction_date': ['2024-01-15', '2024-02-20', '2024-03-10', '2024-04-05', '2024-05-12', '2023-12-25', '2024-06-01', '2024-07-18'],
    'amount': ['1000.50', '200.75', '150.00', '500.25', '300.50', '1200.00', '700.00', '400.00'],
    'currency': ['USD', 'EUR', 'USD', 'USD', 'EUR', 'USD', 'USD', 'EUR'],
    'transaction_type': ['deposit', 'withdrawal', 'deposit', 'withdrawal', 'deposit', 'deposit', 'withdrawal', 'withdrawal']
}

data_clients = {
    'client_id': [101, 102, 103],
    'client_name': ['Иван Иванов', 'Петр Петров', 'Сидор Сидоров'],
    'registration_date': ['2023-12-01', '2024-01-10', '2024-02-15'],
    'client_status': ['active', 'active', 'inactive']
}

transactions = pd.DataFrame(data_transactions)
clients = pd.DataFrame(data_clients)

Объединим данные в один датафрейм для дальнейшего исследования.

In [3]:
df = clients.merge(transactions, on='client_id', how='left')

In [4]:
# Проверим разбиение
transactions.shape, df.shape

((8, 6), (8, 9))

In [5]:
# Просмотрим общую информацию о датафрейме
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   client_id          8 non-null      int64 
 1   client_name        8 non-null      object
 2   registration_date  8 non-null      object
 3   client_status      8 non-null      object
 4   transaction_id     8 non-null      int64 
 5   transaction_date   8 non-null      object
 6   amount             8 non-null      object
 7   currency           8 non-null      object
 8   transaction_type   8 non-null      object
dtypes: int64(2), object(7)
memory usage: 708.0+ bytes


Приведем столбец с суммой транзакций к целочисленному типу, а даты к типу `datetime`.

In [6]:
df['amount'] = df['amount'].astype('float')
df['registration_date'] = pd.to_datetime(df['registration_date'])
df['transaction_date'] = pd.to_datetime(df['transaction_date'])

Отберем только активных клиентов и только данные за 2024 год.

In [7]:
df = df.query('transaction_date >= "2024-01-01" and client_status == "active"')

Теперь найдем для каждого активного клиента:

-	Общую сумму транзакций по типам операций.
-	Среднюю сумму транзакций по типам операций.
-	Количество транзакций по типам операций.
-	Дату последней транзакции.

In [8]:
# Формируем сводную таблицу
df_pivot = df.pivot_table(index=['client_id', 'client_name'], columns='transaction_type',
                          values=['amount', 'transaction_date'],
                          aggfunc={'amount': ['sum', 'mean', 'count'], 'transaction_date':'last'})

In [9]:
# Проверка
df_pivot.head()

amount                                        \
                        count               mean                sum   
transaction_type      deposit withdrawal deposit withdrawal deposit   
client_id client_name                                                 
101       Иван Иванов     2.0        NaN  575.25        NaN  1150.5   
102       Петр Петров     1.0        2.0  300.50    300.375   300.5   

                                 transaction_date             
                                             last             
transaction_type      withdrawal          deposit withdrawal  
client_id client_name                                         
101       Иван Иванов        NaN       2024-03-10        NaT  
102       Петр Петров     600.75       2024-05-12 2024-07-18

In [13]:
# Преобразуем мультииндекс в строки
df_pivot = df_pivot.stack(level='transaction_type', future_stack=True).reset_index().dropna()
# Переименуем столбцы
df_pivot.columns = ['client_id', 'client_name', 'transaction_type', 
                    'transaction_count', 'average_amount', 'total_amount', 'last_transaction_date']

In [21]:
# Проверка
df_pivot = df_pivot.sort_values(by='total_amount', ascending=False)
df_pivot

,client_id,client_name,transaction_type,transaction_count,average_amount,total_amount,last_transaction_date
0,101,Иван Иванов,deposit,2.0,575.250,1150.50,2024-03-10
3,102,Петр Петров,withdrawal,2.0,300.375,600.75,2024-07-18
2,102,Петр Петров,deposit,1.0,300.500,300.50,2024-05-12


Таким образом мы получили итоговый отчет по каждому активному клиенту за 2024 год.

In [23]:
# Запишем результат в файл
df_pivot.to_csv('final_report.csv', index=False)